Failed training, taking too much time to train the model

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
# import the model package
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
# import sklearn

In [3]:
# Load your data
df_train = pd.read_csv('labelled_training_data.csv')
df_valid = pd.read_csv('labelled_validation_data.csv')
df_test = pd.read_csv('labelled_testing_data.csv')

In [2]:
# want to include but not included:
# timestamp, processName(but it probably means the same to processId, and might miss categories), stackAddresses, args

def preprocess(df):
    # Provided by BETH author (Highman 2021)
    df_preprocessed = pd.DataFrame(df[['processId', 'parentProcessId', 'userId', 'mountNamespace', 'eventId', 'returnValue']])

    # Process IDs: Map to 0 for OS processes, else 1
    df_preprocessed['processId'] = df['processId'].map(lambda x: 0 if x in [0, 1, 2] else 1)  # Map to OS

    # Parent Process IDs: Map to 0 for OS processes, else 1
    df_preprocessed['parentProcessId'] = df['parentProcessId'].map(lambda x: 0 if x in [0, 1, 2] else 1)  # Map to OS

    # User IDs: Map to 1 for users with IDs < 1000 (likely OS users), else 0
    df_preprocessed['userId'] = df['userId'].map(lambda x: 1 if x < 1000 else 0)  # Map to OS users

    # Namespace IDs: Check for specific value (402653184) for certain processes, else map to 0
    df_preprocessed['mountNamespace'] = df['mountNamespace'].map(lambda x: 0 if x == 402653184 else 1)  # Check for OS processes

    # Event IDs: Keep event IDs as is (assuming we need to know the maximum value) One-hot coding(?)
    df_preprocessed['eventId'] = df['eventId']

    # Return Value: Map returnValue to 0 for 'failure', 1 for 'success', or 2 for unknown
    df_preprocessed['returnValue'] = df['returnValue'].map(lambda x: 0 if x == 0 else (1 if x == 1 else 2))

    return df_preprocessed

In [7]:
df_train_preprocessed = preprocess(df_train)
df_valid_preprocessed = preprocess(df_valid)
df_test_preprocessed = preprocess(df_test)

In [ ]:
clf = LocalOutlierFactor(n_neighbors = 20, contamination = 0.1)  # contamination = % of outliers in data
predictions = clf.fit_predict(df_train_preprocessed)